# Running cellpose 2.0 on exacloud with a GPU

<font size = 4>Cellpose 2.0 now allows you to train your own models in the GUI!

This notebook allows you to load this **custom model** and run the model on your images with a GPU. 

In this notebook, you can also **train** a custom model using your labels (`_seg.npy`) files, or other labels as `_masks.tif` files. If you already have a trained model, skip this part of the notebook.

For more details on cellpose 2.0 check out the [paper](https://www.biorxiv.org/content/10.1101/2022.04.01.486764v1) or the [talk](https://www.youtube.com/watch?v=3ydtAhfq6H0).


# Setup

We will first check the GPU is working, and mount google drive to get your models and images.

Check CUDA version and that GPU is working in cellpose and import other libraries.

In [10]:
#!nvcc --version
#!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob
from natsort import natsorted


use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

2022-06-06 11:35:05,018 [INFO] ** TORCH CUDA version installed and working. **
>>> GPU activated? YES



#@markdown ###Run this cell to connect your Google Drive to Colab

#@markdown * Click on the URL. 

#@markdown * Sign in your Google Account. 

#@markdown * Copy the authorization code. 

#@markdown * Enter the authorization code. 

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive". 

#mounts user's Google Drive to Google Colab.

from google.colab import drive
drive.mount('/content/gdrive')




### Download sample images (optional)

If you don't mount your google drive, and want to test cellpose 2.0, run the next code block to download the example data. This `human_in_the_loop` folder has a `train` folder with training images and manual segmentations (in this case created in the loop), and a `test` folder with test images and manual segmentations from scratch.

These images are from the breast vectra class from the Tissuenet dataset. The full Tissuenet dataset is available [here](https://datasets.deepcell.org/), and for a description of the data see the [paper](https://www.nature.com/articles/s41587-021-01094-0).

We will convert the `_seg.npy` files to `_masks.tif` files.

In [ ]:
# !rm -rf human_in_the_loop/

train_files = natsorted(glob('human_in_the_loop/train/*.tif'))
train_seg = natsorted(glob('human_in_the_loop/train/*.npy'))

test_files = natsorted(glob('human_in_the_loop/test/*.npy'))
test_seg = natsorted(glob('human_in_the_loop/test/*.npy'))


what the training images look like + their labels

plt.figure(figsize=(12,4), dpi=300)
for k,f in enumerate(train_files):
    img = io.imread(f)
    plt.subplot(2,len(train_files),k+1)
    img = np.vstack((img, np.zeros_like(img)[:1]))
    img = img.transpose(1,2,0)
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(2,len(train_files),len(train_files) + k+1)
    seg = np.load(os.path.splitext(f)[0] + '_seg.npy', allow_pickle=True).item()
    masks= seg['masks'].squeeze()
    plt.imshow(masks)
    plt.axis('off')


# Train model on manual annotations

Skip this step if you already have a pretrained model.

Fill out the form below with the paths to your data and the parameters to start training.

## Training parameters

<font size = 4> **Paths for training, predictions and results**


<font size = 4>**`train_dir:`, `test_dir`:** These are the paths to your folders train_dir (with images and masks of training images) and test_dir (with images and masks of test images). You can leave the test_dir blank, but it's recommended to have some test images to check the model's performance. To find the paths of the folders containing the respective datasets, go to your Files on the left of the notebook, navigate to the folder containing your files and copy the path by right-clicking on the folder, **Copy path** and pasting it into the right box below.

<font size = 4>**`initial_model`:** Choose a model from the cellpose [model zoo](https://cellpose.readthedocs.io/en/latest/models.html#model-zoo) to start from.

<font size = 4>**`model_name`**: Enter the path where your model will be saved once trained (for instance your result folder).

<font size = 4>**Training parameters**

<font size = 4>**`number_of_epochs`:** Input how many epochs (rounds) the network will be trained. At least 100 epochs are recommended, but sometimes 250 epochs are necessary, particularly from scratch. **Default value: 100**



In [49]:
model_name = "Ctn"

#@markdown ###Path to images and masks:
data_path = "/home/exacloud/gscratch/HeiserLab/images/cellpose_training"
train_dir = os.path.join(data_path, "train") #@param {type:"string"}
test_dir = os.path.join(data_path, "test") #@param {type:"string"}
#Define where the patch file will be saved
base = "/content"

# model name and path
#@markdown ###Name of the pretrained model to start from and new model name:
from cellpose import models
initial_model = "Ctn" #@param ['cyto','nuclei','tissuenet','livecell','cyto2','CP','CPx','TN1','TN2','TN3','LC1','LC2','LC3','LC4','scratch']
model_name = "Ctn" #@param {type:"string"}

# other parameters for training.
#@markdown ###Training Parameters:
#@markdown Number of epochs:
n_epochs =  100#@param {type:"number"}

Channel_to_use_for_training = "Grayscale" #@param ["Grayscale", "Blue", "Green", "Red"]

# @markdown ###If you have a secondary channel that can be used for training, for instance nuclei, choose it here:

Second_training_channel= "None" #@param ["Grayscale", "Blue", "Green", "Red"]


#@markdown ###Advanced Parameters

Use_Default_Advanced_Parameters = True #@param {type:"boolean"}
#@markdown ###If not, please input:
learning_rate = 0.1 #@param {type:"number"}
weight_decay = 0.0001 #@param {type:"number"}

if (Use_Default_Advanced_Parameters): 
  print("Default advanced parameters enabled")
  learning_rate = 0.1 
  weight_decay = 0.0001
  
#here we check that no model with the same name already exist, if so delete
model_path = train_dir + 'models/'
if os.path.exists(model_path+'/'+model_name):
  print("!! WARNING: "+model_name+" already exists and will be deleted in the following cell !!")
  
if len(test_dir) == 0:
  test_dir = None

# Here we match the channel to number
if Channel_to_use_for_training == "Grayscale":
  chan = 0
elif Channel_to_use_for_training == "Blue":
  chan = 3
elif Channel_to_use_for_training == "Green":
  chan = 2
elif Channel_to_use_for_training == "Red":
  chan = 1


if Second_training_channel == "Blue":
  chan2 = 3
elif Second_training_channel == "Green":
  chan2 = 2
elif Second_training_channel == "Red":
  chan2 = 1
elif Second_training_channel == "None":
  chan2 = 0

if initial_model=='scratch':
  initial_model = 'None'

Default advanced parameters enabled


Here's what the command to train would be on the command line -- make sure if you run this locally to correct the paths for your local computer.

In [26]:
run_str = f'python -m cellpose --use_gpu --verbose --train --dir {train_dir} --pretrained_model {initial_model} --chan {chan} --chan2 {chan2} --n_epochs {n_epochs} --learning_rate {learning_rate} --weight_decay {weight_decay}'
if test_dir is not None:
    run_str += f' --test_dir {test_dir}'
run_str += ' --mask_filter _seg.npy' # if you want to use _seg.npy files for training
print(run_str)

python -m cellpose --use_gpu --verbose --train --dir /home/exacloud/gscratch/HeiserLab/images/cellpose_training/train --pretrained_model cyto2 --chan 0 --chan2 1 --n_epochs 100 --learning_rate 0.1 --weight_decay 0.0001 --test_dir /home/exacloud/gscratch/HeiserLab/images/cellpose_training/test --mask_filter _seg.npy


## Train new model

Using settings from form above, train model in notebook.

In [64]:
model_path = os.path.join(data_path, "train/models",model_name) #@param {type:"string"}

# start logger (to see training across epochs)
logger = io.logger_setup()

# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path,
                            net_avg = True,
                            diam_mean = 15.3)
# set channels
channels = [chan, chan2]

# get files
output = io.load_train_test_data(train_dir, mask_filter='_seg.npy')
train_data, train_labels, _, test_data, test_labels, _ = output

new_model_path = model.train(train_data, train_labels, 
                              #test_data=test_data,
                              #test_labels=test_labels,
                              channels=channels,
                              save_path=train_dir, 
                              n_epochs=n_epochs,
                              learning_rate=learning_rate, 
                              weight_decay=weight_decay, 
                              nimg_per_epoch=8,
                              model_name=model_name)

# diameter of labels in training images
diam_labels = model.diam_labels.copy()

2022-06-06 14:22:58,429 [INFO] WRITING LOG OUTPUT TO /home/users/dane/.cellpose/run.log
2022-06-06 14:22:58,431 [INFO] >>>> loading model /home/exacloud/gscratch/HeiserLab/images/cellpose_training/train/models/Ctn
2022-06-06 14:22:58,432 [INFO] ** TORCH CUDA version installed and working. **
2022-06-06 14:22:58,434 [INFO] >>>> using GPU
2022-06-06 14:22:58,571 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2022-06-06 14:22:58,572 [INFO] >>>> model diam_labels =  15.371 (mean diameter of training ROIs)
2022-06-06 14:22:58,578 [INFO] not all flows are present, running flow generation for all images
2022-06-06 14:22:58,713 [INFO] 10 / 10 images in /home/exacloud/gscratch/HeiserLab/images/cellpose_training/train folder have labels
2022-06-06 14:22:59,122 [INFO] computing flows for labels


100%|██████████| 10/10 [00:00<00:00, 16.26it/s]


2022-06-06 14:23:00,169 [INFO] >>>> median diameter set to = 30
2022-06-06 14:23:00,171 [INFO] >>>> mean of training label mask diameters (saved to model) 15.371
2022-06-06 14:23:00,171 [INFO] >>>> training network with 2 channel input <<<<
2022-06-06 14:23:00,172 [INFO] >>>> LR: 0.10000, batch_size: 8, weight_decay: 0.00010
2022-06-06 14:23:00,173 [INFO] >>>> ntrain = 10
2022-06-06 14:23:00,174 [INFO] >>>> nimg_per_epoch = 10
2022-06-06 14:23:00,514 [INFO] Epoch 0, Time  0.3s, Loss 0.0791, LR 0.0000
2022-06-06 14:23:00,789 [INFO] saving network parameters to /home/exacloud/gscratch/HeiserLab/images/cellpose_training/train/models/Ctn
2022-06-06 14:23:01,990 [INFO] Epoch 5, Time  1.8s, Loss 0.0780, LR 0.0556
2022-06-06 14:23:03,420 [INFO] Epoch 10, Time  3.2s, Loss 0.0626, LR 0.1000
2022-06-06 14:23:06,233 [INFO] Epoch 20, Time  6.1s, Loss 0.0650, LR 0.1000
2022-06-06 14:23:09,075 [INFO] Epoch 30, Time  8.9s, Loss 0.0544, LR 0.1000
2022-06-06 14:23:11,899 [INFO] Epoch 40, Time 11.7s, Lo

## Evaluate on test data (optional)

If you have test data, check performance

In [68]:
# get files (during training, test_data is transformed so we will load it again)
output = io.load_train_test_data(test_dir, mask_filter='_seg.npy')
test_data, test_labels = output[:2]

# run model on test images
masks = model.eval(test_data, 
                   channels=[chan, chan2],
                   diameter=diam_labels)[0]

# check performance using ground truth labels
ap = metrics.average_precision(test_labels, masks)[0]
print('')
print(f'>>> average precision at iou threshold 0.5 = {ap[:,0].mean():.3f}')


2022-06-06 14:25:15,038 [INFO] not all flows are present, running flow generation for all images
2022-06-06 14:25:15,062 [INFO] 1 / 1 images in /home/exacloud/gscratch/HeiserLab/images/cellpose_training/test folder have labels

>>> average precision at iou threshold 0.5 = 0.661


plot masks

In [67]:
train_files = natsorted(glob(train_dir))
plt.figure(figsize=(12,8), dpi=150)
for k,im in enumerate(test_data):
    img = im.copy()
    plt.subplot(3,len(train_files), k+1)
    img = np.vstack((img, np.zeros_like(img)[:1]))
    img = img.transpose(1,2,0)
    plt.imshow(img)
    plt.axis('off')
    if k==0:
        plt.title('image')

    plt.subplot(3,len(train_files), len(train_files) + k+1)
    plt.imshow(masks[k])
    plt.axis('off')
    if k==0:
        plt.title('predicted labels')

    plt.subplot(3,len(train_files), 2*len(train_files) + k+1)
    plt.imshow(test_labels[k])
    plt.axis('off')
    if k==0:
        plt.title('true labels')
plt.tight_layout()

TypeError: 'NoneType' object is not iterable

<Figure size 1800x1200 with 0 Axes>

# Use custom model to segment images

Take custom trained model from above

## Parameters

In [44]:
# model name and path

#@markdown ###Custom model path (full path):

model_path = os.path.join(data_path, "train/models",model_name) #@param {type:"string"}

#@markdown ###Path to images:

dir = os.path.join(data_path, "image_cache") #@param {type:"string"}


#@markdown ###Channel Parameters:

Channel_to_use_for_segmentation = "Grayscale" #@param ["Grayscale", "Blue", "Green", "Red"]

# @markdown If you have a secondary channel that can be used, for instance nuclei, choose it here:

Second_segmentation_channel= "None" #@param ["None", "Blue", "Green", "Red"]


# Here we match the channel to number
if Channel_to_use_for_segmentation == "Grayscale":
  chan = 0
elif Channel_to_use_for_segmentation == "Blue":
  chan = 3
elif Channel_to_use_for_segmentation == "Green":
  chan = 2
elif Channel_to_use_for_segmentation == "Red":
  chan = 1


if Second_segmentation_channel == "Blue":
  chan2 = 3
elif Second_segmentation_channel == "Green":
  chan2 = 2
elif Second_segmentation_channel == "Red":
  chan2 = 1
elif Second_segmentation_channel == "None":
  chan2 = 0

#@markdown ### Segmentation parameters:

#@markdown diameter of cells (set to zero to use diameter from training set):
diameter =  0#@param {type:"number"}
#@markdown threshold on flow error to accept a mask (set higher to get more cells, e.g. in range from (0.1, 3.0), OR set to 0.0 to turn off so no cells discarded):
flow_threshold = 0.4 #@param {type:"slider", min:0.0, max:3.0, step:0.1}
#@markdown threshold on cellprob output to seed cell masks (set lower to include more pixels or higher to include fewer, e.g. in range from (-6, 6)):
cellprob_threshold=0 #@param {type:"slider", min:-6, max:6, step:1}


if you're using the example test data we'll copy it to a new folder

src = 'human_in_the_loop/test'
if dir[:len(src)] == src:
    files = io.get_image_files(dir, '_masks')
    dir = 'human_in_the_loop/eval/'
    os.makedirs(dir, exist_ok=True)
    for f in files:
        dst = dir + os.path.split(f)[1]
        print(f'{f} > {dst}')
        shutil.copyfile(f, dst)

Here's what the command to train would be on the command line -- make sure if you run this locally to correct the paths for your local computer.

In [ ]:
run_str = f'python -m cellpose --use_gpu --verbose --dir {dir} --pretrained_model {model_path} --chan {chan} --chan2 {chan2} --diameter {diameter} --flow_threshold {flow_threshold} --cellprob_threshold {cellprob_threshold}'
print(run_str)

python -m cellpose --use_gpu --verbose --dir human_in_the_loop/eval/ --pretrained_model human_in_the_loop/train/models/CP_tissuenet --chan 2 --chan2 1 --diameter 0 --flow_threshold 0.4 --cellprob_threshold 0


## run custom model

how to run the custom model in a notebook

In [ ]:
# gets image files in dir (ignoring image files ending in _masks)
files = io.get_image_files(dir, '_masks', imf = "np_img")
print(files)
images = [io.imread(f) for f in files]

# declare model
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

# use model diameter
diameter = model.diam_labels 

# run model on test images
masks, flows, styles = model.eval(images, 
                                  channels=[chan, chan2],
                                  diameter=diameter,
                                  flow_threshold=flow_threshold,
                                  cellprob_threshold=cellprob_threshold
                                  )

['/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00701_A1_1_004_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00701_A1_1_188_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00701_D2_1_188_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00701_D4_1_188_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00701_D6_1_188_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00801_A1_1_004_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00801_A1_1_188_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00801_D2_1_188_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00801_D4_1_188_np_img.tif', '/home/exacloud/gscratch/HeiserLab/images/cellpose_training/image_cache/HC00801_D

## save output to *_seg.npy

you will see the files save in the Files tab and you can download them from there

In [ ]:
from cellpose import io

io.masks_flows_to_seg(images, 
                      masks, 
                      flows, 
                      diameter*np.ones(len(masks)), 
                      files, 
                      channels)

## save output masks to tiffs/pngs or txt files for imageJ

In [ ]:
io.save_masks(images, 
              masks, 
              flows, 
              files, 
              channels=channels,
              png=True, # save masks as PNGs and save example image
              tif=True, # save masks as TIFFs
              save_txt=True, # save txt outlines for ImageJ
              save_flows=False, # save flows as TIFFs
              save_outlines=False, # save outlines as TIFFs 
              )
    

In [ ]:
f = files[0]
plt.figure(figsize=(12,4), dpi=300)
plt.imshow(io.imread(os.path.splitext(f)[0] + '_cp_masks.png'))
plt.axis('off')